In [11]:
# data만 tensorflow를 이용해 load -> numpy 객체로 받아옴
import numpy as np
import tensorflow as tf
from tensorflow import keras

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [12]:
import torch
from torch import nn
from torch.nn import functional as F

from skorch import NeuralNetClassifier

In [13]:
class MLP_layers(nn.Module):
    def __init__(self, input_size, n_hidden, n_neurons):
        super().__init__()
        layers = [nn.Linear(input_size, n_neurons), nn.ReLU()]
        for _ in range(n_hidden-1):
            layers.append(nn.Linear(n_neurons,n_neurons))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(n_neurons, 10))
        self.layers = nn.Sequential(*layers)
    def forward(self, x):
        # if len(x.shape) == 3:
        #     x = x.unsqueeze(1)
        x = torch.flatten(x,1)
        x = self.layers(x)
        return x

In [14]:
# 딥러닝이기 때문에 CV를 사용하지 않고, hold out을 사용
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit

# hold out instead of cross-validation
valid_idx = [-1]*X_train.shape[0] +[0]*X_valid.shape[0]
X_train_valid = np.concatenate([X_train, X_valid])
y_train_valid = np.concatenate([y_train, y_valid])
ps = PredefinedSplit(valid_idx)

In [15]:
from scipy import stats
param_random = {
    'module__n_hidden':stats.randint(0,8+1),
    'module__n_neurons':stats.randint(16,256+1),
    'lr':stats.loguniform(1e-4, 1e-2),
    'optimizer':[torch.optim.SGD, torch.optim.Adam]
}
net = NeuralNetClassifier(MLP_layers,
                            module__input_size = 28*28,
                            module__n_hidden=2,
                            module__n_neurons=16,
                            max_epochs = 10,
                            lr = 1e-2,
                            optimizer = torch.optim.SGD,
                            criterion = nn.CrossEntropyLoss,
                            device = 'cuda',
                            train_split=None,
                            verbose=0
                            )
rand = RandomizedSearchCV(net, param_random, cv=ps, n_jobs=-1, n_iter=15)

In [16]:
rand = RandomizedSearchCV(net, param_random, cv=ps, n_jobs=-1, n_iter=15)
rand.fit(torch.Tensor(X_train_valid), torch.Tensor(y_train_valid).type(torch.LongTensor))

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.MLP_layers'>,
  module__input_size=784,
  module__n_hidden=2,
  module__n_neurons=16,
),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'lr': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f443307bee0>,
                                        'module__n_hidden': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f443307bc10>,
                                        'module__n_neurons': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f443307bf40>,
                                        'optimizer': [<class 'torch.optim.sgd.SGD'>,
                                                      <class 'torch.optim.adam.Adam'>]})

In [19]:
import pandas as pd

pd.DataFrame(rand.cv_results_).sort_values(by='split0_test_score',ascending=False)[['mean_fit_time',
'param_lr','param_module__n_hidden','param_module__n_neurons','param_optimizer','split0_test_score','mean_test_score']].head()

,mean_fit_time,param_lr,param_module__n_hidden,param_module__n_neurons,param_optimizer,split0_test_score,mean_test_score
8,31.077406,0.000217,6,155,<class 'torch.optim.adam.Adam'>,0.8772,0.8772
3,27.965037,0.001078,4,38,<class 'torch.optim.adam.Adam'>,0.8698,0.8698
5,28.068388,0.002493,3,236,<class 'torch.optim.adam.Adam'>,0.8688,0.8688
13,28.239262,0.002234,4,253,<class 'torch.optim.adam.Adam'>,0.8660,0.8660
14,28.242951,0.002857,6,145,<class 'torch.optim.sgd.SGD'>,0.8638,0.8638
